In [47]:
import pandas as pd
import os
import numpy as np

# Get all file paths in the Nutritional_value_Data folder
paths = [
    os.path.join("raw_data", filename)
    for filename in os.listdir("raw_data")
    if os.path.isfile(os.path.join("raw_data", filename))
]

print(paths)

['raw_data/nutrient_value_some_common_foods_soups-2008-valeur_nutritive_quelques_aliments_usuels_soupe-eng.csv', 'raw_data/nutrient_value_miscellaneous_foods_2008_valeur_nutritive_divers-eng.csv', 'raw_data/nutrient_value_dairy_foods_and_other_related_products_2008_valeur_nutritive_produits_laitiers_et_produits_annexes-eng.csv', 'raw_data/nutrient_value_legumes_nuts_and_seeds_2008_valeur_nutritive_legumineuses_noix_et_graines-eng.csv', 'raw_data/nutrient_value_meat_and_poultry_2008_valeur_nutritive_2008_viande_et_volaille-eng.csv', 'raw_data/nutrient_value_vegetables_and_vegetable_products_2008_valeur_nutritive_legumes_et_produits_base_de_legumes-eng.csv', 'raw_data/nutrient_value_some_common_foods_sweets_sugars-2008-valeur_nutritive_quelques_aliments_usuels_sucres_sucreries-eng.csv', 'raw_data/nutrient_value_mixed_dishes_2008_valeur_nutritive_mets_composes-eng.csv', 'raw_data/nutrient_value_fats_and_oils_foods_2008_valeur_nutritive_matieres_grasses_et_huiles-eng.csv', 'raw_data/nutrie

In [48]:
# Leave only columns that are present in all the files
# The data is not very clean, and usually there are two cases how to clearn the data
cols_all = set()

for path in paths:
    df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
    df = df.dropna(how='all')
    if df.iloc[1, 0] == 'Food name':
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        cols_all = cols_all.union(cols)
    elif df.iloc[2, 0] == 'Food name':
        df = df.iloc[1:, :]
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[3:].reset_index(drop=True)
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        cols_all = cols_all.union(cols)
common_cols = list(cols_all)
common_cols.insert(0, common_cols.pop(common_cols.index('Food Category')))
common_cols.insert(1, common_cols.pop(common_cols.index('Food name')))
common_cols.insert(2, common_cols.pop(common_cols.index('Weight ')))
common_cols.insert(3, common_cols.pop(common_cols.index('Measure')))
common_cols.insert(4, common_cols.pop(common_cols.index('Protein')))
common_cols.insert(5, common_cols.pop(common_cols.index('Carbohydrate')))
common_cols.insert(6, common_cols.pop(common_cols.index('Total Fat')))
common_cols

['Food Category',
 'Food name',
 'Weight ',
 'Measure',
 'Protein',
 'Carbohydrate',
 'Total Fat',
 'Protei',
 'Magnesium',
 'Folate ',
 'Monounsaturated fat',
 'Folate',
 'Beta carotene',
 'Trans Fat',
 'Total Dietary Fibre',
 'Potassium',
 'Total Sugar',
 'Polyunsaturated Fat',
 'Thiamin',
 'EPA',
 'Polyunsaturated fat',
 'Riboflavin',
 'Vitamin D',
 'DHA',
 'Calcium',
 'Cholesterol',
 'Vitamin C',
 'Monounsaturated Fat',
 'Alcohol',
 'Energy',
 'Iron',
 'Sodium',
 'Vitamin B12',
 'Total Daietary Fibre',
 'Lycopene',
 'Trans fats',
 'Vitamin E',
 'Niacin',
 'Phosphorus',
 'Vitamin A',
 'Dietary Folate Equivalents',
 ' Folate',
 'Saturated Fat',
 'Caffeine']

In [49]:
# Combine all the individual files into one big dataframe
# The data is not very clean, and usually there are two cases how to clearn the data
big_df = pd.DataFrame()
problematic_paths = []
for path in paths:
    try:
        df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
        df = df.dropna(how='all')
        if df.iloc[1, 0] == 'Food name':
            #print('good (case 1)')
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
            df = df.iloc[1:, :]
            subcategory = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df['Food Category'] = category
            print('Category: ', category)
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            for col in common_cols:
                if col not in cols:
                    df[col] = 'N/A'
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        elif df.iloc[2, 0] == 'Food name':
            #print('good (case 2)')
            df = df.iloc[1:, :]
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[3:].reset_index(drop=True)
            df['Food Category'] = category
            print('Category: ', category)
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            for col in common_cols:
                if col not in cols:
                    df[col] = 'N/A'
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        else:
            print('error')
            problematic_paths.append(path)
    except Exception as e:
        print(str(e))
        problematic_paths.append(path)

Category:  SOUPS
Category:  MISCELLANEOUS FOODS
Category:  DAIRY FOODS AND OTHER RELATED PRODUCTS
Category:  LEGUMES, NUTS AND SEEDS
Category:  MEAT AND POULTRY
Category:  VEGETABLES AND VEGETABLE PRODUCTS
Category:  SWEETS AND SUGARS
Category:  MIXED DISHES
Category:  FATS AND OILS
Category:  BAKED GOODS
Category:  FRUIT AND FRUIT JUICES
Category:  FAST FOODS
Category:  FISH AND SHELLFISH
Category:  BEVERAGES
Category:  BREADS, CEREALS AND OTHER GRAIN PRODUCTS
Category:  EGGS AND EGG DISHES
Category:  SNACKS


In [50]:
problematic_paths

[]

In [51]:
big_df

,Food Category,Food name,Weight,Measure,Protein,Carbohydrate,Total Fat,Protei,Magnesium,Folate,...,Lycopene,Trans fats,Vitamin E,Niacin,Phosphorus,Vitamin A,Dietary Folate Equivalents,Folate,Saturated Fat,Caffeine
0,SOUPS,"Beef or chicken, broth/bouillon",254,250mL,3,tr,1,N/A,5,5,...,N/A,N/A,N/A,N/A,33,0,N/A,N/A,0.3,N/A
1,SOUPS,"Beef, chunky",254,250mL,12,21,5,N/A,5,15,...,N/A,N/A,N/A,N/A,127,137,N/A,N/A,2.7,N/A
2,SOUPS,"Chicken noodle, chunky",254,250mL,13,18,6,N/A,10,65,...,N/A,N/A,N/A,N/A,76,71,N/A,N/A,1.5,N/A
3,SOUPS,"Chicken noodle, low fat, reduced salt",257,250mL,9,13,2,N/A,NaN,NaN,...,N/A,N/A,N/A,N/A,NaN,NaN,N/A,N/A,1.0,N/A
4,SOUPS,"Chicken vegetable, chunky",254,250mL,13,20,5,N/A,10,13,...,N/A,N/A,N/A,N/A,112,317,N/A,N/A,1.5,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,SNACKS,"Rice cakes, plain",9,1,1,7,tr,N/A,12,N/A,...,N/A,N/A,N/A,N/A,32,0,N/A,N/A,0.1,N/A
21,SNACKS,"Sesame sticks, salted",14,60mL,2,6,5,N/A,6,N/A,...,N/A,N/A,N/A,N/A,19,0,N/A,N/A,0.9,N/A
22,SNACKS,"Soybeans, roasted, salted",44,60mL,15,15,11,N/A,63,N/A,...,N/A,N/A,N/A,N/A,158,4,N/A,N/A,1.6,N/A
23,SNACKS,"Trail mix, regular",38,60mL,5,17,11,N/A,60,N/A,...,N/A,N/A,N/A,N/A,131,tr,N/A,N/A,2.1,N/A


In [52]:
cols_to_check = [col for col in big_df.columns if col not in ['Food Category', 'Food name']]
big_df = big_df.dropna(subset=cols_to_check, how='all')
big_df = big_df.reset_index(drop=True)
big_df

,Food Category,Food name,Weight,Measure,Protein,Carbohydrate,Total Fat,Protei,Magnesium,Folate,...,Lycopene,Trans fats,Vitamin E,Niacin,Phosphorus,Vitamin A,Dietary Folate Equivalents,Folate,Saturated Fat,Caffeine
0,SOUPS,"Beef or chicken, broth/bouillon",254,250mL,3,tr,1,N/A,5,5,...,N/A,N/A,N/A,N/A,33,0,N/A,N/A,0.3,N/A
1,SOUPS,"Beef, chunky",254,250mL,12,21,5,N/A,5,15,...,N/A,N/A,N/A,N/A,127,137,N/A,N/A,2.7,N/A
2,SOUPS,"Chicken noodle, chunky",254,250mL,13,18,6,N/A,10,65,...,N/A,N/A,N/A,N/A,76,71,N/A,N/A,1.5,N/A
3,SOUPS,"Chicken noodle, low fat, reduced salt",257,250mL,9,13,2,N/A,NaN,NaN,...,N/A,N/A,N/A,N/A,NaN,NaN,N/A,N/A,1.0,N/A
4,SOUPS,"Chicken vegetable, chunky",254,250mL,13,20,5,N/A,10,13,...,N/A,N/A,N/A,N/A,112,317,N/A,N/A,1.5,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,SNACKS,"Rice cakes, plain",9,1,1,7,tr,N/A,12,N/A,...,N/A,N/A,N/A,N/A,32,0,N/A,N/A,0.1,N/A
1157,SNACKS,"Sesame sticks, salted",14,60mL,2,6,5,N/A,6,N/A,...,N/A,N/A,N/A,N/A,19,0,N/A,N/A,0.9,N/A
1158,SNACKS,"Soybeans, roasted, salted",44,60mL,15,15,11,N/A,63,N/A,...,N/A,N/A,N/A,N/A,158,4,N/A,N/A,1.6,N/A
1159,SNACKS,"Trail mix, regular",38,60mL,5,17,11,N/A,60,N/A,...,N/A,N/A,N/A,N/A,131,tr,N/A,N/A,2.1,N/A


In [53]:
big_df.to_csv('Food_all.csv', index=False)